In [1]:
import os,sys
import numpy as np
from collections import namedtuple
import tqdm
import glob
import math
import random
import inspect
import os.path as osp
from pathlib import Path
import itertools
from itertools import chain
import numpy as np
import pandas as pd
import multiprocessing
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sys, os
from importlib import reload

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.data import Data, DataLoader, DataListLoader
from torch_geometric.nn import EdgeConv, global_mean_pool, DataParallel
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data,Dataset
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv
import h5py

In [94]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../')))

import models_torch.models as models
reload(models)
import models_torch.losses as losses
reload(losses)
import utils_torch.scaler
reload(utils_torch.scaler)
import utils_torch.preprocessing as prepr
import utils_torch.plot_util as plot
reload(plot)
import utils_torch.train_util as train
reload(train)
import graph_data.graph_data as graph_data
reload(graph_data)
import utils_torch.model_summary as summary
reload(summary)

sys.path.append(os.path.abspath(os.path.join('../../')))
import vande.util.util_plotting as vande_plot
reload(vande_plot)

sys.path.append(os.path.abspath(os.path.join('../../')))
sys.path.append(os.path.abspath(os.path.join('../../DarkFlow/darkflow/')))
import DarkFlow.darkflow.networks.flows as flows


In [3]:
data_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
dataset = graph_data.GraphDataset(root=data_dir,n_jets=1e3)
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
   
use_generator = False
if use_generator:
    validation_split = 0.2
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    if dataset_size > 2:
        split = int(np.floor(validation_split * dataset_size))
    else: 
        split = 1
    print(dataset_size,split)
    random_seed= 1001

    train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
                                                             generator=torch.Generator().manual_seed(random_seed))
    print("train subset dim:", len(train_subset))
    print("validation subset dim", len(val_subset))
    dataloaders = {
        'train':  DataLoader(train_subset, batch_size=128, shuffle=True),
        'val':   DataLoader(val_subset, batch_size=128, shuffle=True)
    }
    print("train_dataloader dim:", len(dataloaders['train']))
    print("val dataloader dim:", len(dataloaders['val']))
else : 
    in_memory_datas = dataset.return_inmemory_data() 

In [ ]:
pf_cands,jet_prop = dataset.pf_cands,dataset.jet_prop
pf_cands_t = torch.cat([torch.tensor(pf_cands[i], dtype=torch.float) for i in range(len(pf_cands))])
#Plot consistuents and jet features prepared for the graph! (but before any normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
jet_feats = 'N_constituents,M,Pt,Eta,Phi'.split(',')


save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
vande_plot.plot_features(pf_cands_t.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)
vande_plot.plot_features(jet_prop[:,0:-1], jet_feats ,'Normalized' , 'QCD', plotname='{}plot_jet_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [4]:
#scaler = standardize(train_subset) # I dont think that this works for the dataset implementation as it is done now, implement scaler as part of dataset ? 
scaler = prepr.standardize(in_memory_datas,minmax_idx=[3,4],log_idx=[3,4]) 

dataloaders = {
    'train':  DataLoader(in_memory_datas, batch_size=128, shuffle=True)
    }
print("train_dataloader dim:", len(dataloaders['train']))

#dataset.get(0).u

train_dataloader dim: 8


In [ ]:
pf_cands_norm = torch.cat([torch.tensor(in_memory_datas[i].x, dtype=torch.float) for i in range(len(in_memory_datas))])
#Plot consistuents and jet features prepared for the graph! (after normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats_normalized/'
vande_plot.plot_features(pf_cands_norm.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [5]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
multi_gpu = False #torch.cuda.device_count()>1

In [95]:
# model
input_dim = 7
output_dim = 7#4#4,7
big_dim = 32
hidden_dim = 2
#model = models.EdgeNet(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)
#model = models.EdgeNetVAE(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)
model = models.PlanarEdgeNetVAE(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = 10e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, threshold=1e-6)

model.to(device)
print(model)
summary.gnn_model_summary(model)

PlanarEdgeNetVAE(
  (mu_layer): Linear(in_features=32, out_features=2, bias=True)
  (var_layer): Linear(in_features=32, out_features=2, bias=True)
  (batchnorm): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  ))
  (decoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=7, bias=True)
  ))
  (amor_u): Linear(in_features=32, out_features=12, bias=True)
  (amor_w): Linear(in_features=32, out_features=12, bias=True)
  (amor_b): Linear(in_features=32, out_features=6, bias=True)
  (flow_0): Planar(
    (h): Tanh()
    (softplus): Softplus(beta=1, threshold=20)
  )
  (flow_1): Planar(
    (h): Tanh()
  

In [96]:
#loss
#loss_ftn_obj = losses.LossFunction('mse_coordinates', device=device)
#loss_ftn_obj = losses.LossFunction('mse', device=device)
loss_ftn_obj = losses.LossFunction('vae_loss_mse',beta=0.00001,device=device)

# Training loop
n_epochs = 80
stale_epochs = 0
loss = 999999
#train_losses = []
for epoch in range(0, n_epochs):
    #loss = train(model, optimizer, loader, len(datas), 128, loss_ftn_obj)
    loss_tot,loss_reco,loss_kl = train.train(model, optimizer, dataloaders['train'], len(dataloaders['train'].dataset), dataloaders['train'].batch_size, loss_ftn_obj)
    #train_losses.append(loss)
    print('Epoch: {:02d}, Training Loss Tot, Reco, KL :  {:.4f},{:.4f}, {:.4f}'.format(epoch, loss_tot,loss_reco,loss_kl))













  0%|          | 0/7.8125 [00:00<?, ?it/s]











train loss tot = 0.6902522,train loss reco = 0.6877825,train loss kl = 247.6687317:   0%|          | 0/7.8125 [00:00<?, ?it/s]











train loss tot = 0.6902522,train loss reco = 0.6877825,train loss kl = 247.6687317:   0%|          | 0/7.8125 [00:00<?, ?it/s]











train loss tot = 0.6902522,train loss reco = 0.6877825,train loss kl = 247.6687317:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.43it/s]











train loss tot = 0.7031828,train loss reco = 0.7017013,train loss kl = 148.8526917:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.43it/s]











train loss tot = 0.7031828,train loss reco = 0.7017013,train loss kl = 148.8526917:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.43it/s]











train loss tot = 0.7031828,train loss reco = 0.7017013,train loss kl = 148.8526917:  26%|██▌       | 2/7.8125 [00:00<00:00,  7.20it/s]











train loss tot = 0.7861138,train loss reco = 0.7852523,train loss kl = 86.

Epoch: 00, Training Loss Tot, Reco, KL :  0.7238,0.7224, 134.3575














train loss tot = 0.5896250,train loss reco = 0.5861740,train loss kl = 345.6887207:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.26it/s]











train loss tot = 0.5896250,train loss reco = 0.5861740,train loss kl = 345.6887207:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.26it/s]











train loss tot = 0.6504185,train loss reco = 0.6461112,train loss kl = 431.3707581:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.26it/s]











train loss tot = 0.6504185,train loss reco = 0.6461112,train loss kl = 431.3707581:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.26it/s]











train loss tot = 0.6504185,train loss reco = 0.6461112,train loss kl = 431.3707581:  51%|█████     | 4/7.8125 [00:00<00:00, 10.26it/s]











train loss tot = 0.5939522,train loss reco = 0.5892915,train loss kl = 466.6555176:  51%|█████     | 4/7.8125 [00:00<00:00, 10.26it/s]











train loss tot = 0.5939522,train loss reco = 0.5892915,train loss kl = 466.6555176:  51%|█████     | 4/7.8125 [0

Epoch: 01, Training Loss Tot, Reco, KL :  0.6147,0.6099, 480.2078














train loss tot = 0.4354781,train loss reco = 0.4249387,train loss kl = 1054.3663330:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.47it/s]











train loss tot = 0.4354781,train loss reco = 0.4249387,train loss kl = 1054.3663330:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.47it/s]











train loss tot = 0.4961973,train loss reco = 0.4846436,train loss kl = 1155.8510742:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.47it/s]











train loss tot = 0.4961973,train loss reco = 0.4846436,train loss kl = 1155.8510742:  26%|██▌       | 2/7.8125 [00:00<00:00, 10.47it/s]











train loss tot = 0.4961973,train loss reco = 0.4846436,train loss kl = 1155.8510742:  51%|█████     | 4/7.8125 [00:00<00:00, 10.51it/s]











train loss tot = 0.4683457,train loss reco = 0.4575359,train loss kl = 1081.4409180:  51%|█████     | 4/7.8125 [00:00<00:00, 10.51it/s]











train loss tot = 0.4683457,train loss reco = 0.4575359,train loss kl = 1081.4409180:  51%|█████     | 4/7.

...

Epoch: 77, Training Loss Tot, Reco, KL :  0.0481,0.0470, 107.3076














train loss tot = 0.0569317,train loss reco = 0.0557356,train loss kl = 119.6636505:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.54it/s]











train loss tot = 0.0569317,train loss reco = 0.0557356,train loss kl = 119.6636505:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.54it/s]











train loss tot = 0.0569317,train loss reco = 0.0557356,train loss kl = 119.6636505:  26%|██▌       | 2/7.8125 [00:00<00:00,  6.60it/s]











train loss tot = 0.0367194,train loss reco = 0.0356031,train loss kl = 111.6627045:  26%|██▌       | 2/7.8125 [00:00<00:00,  6.60it/s]











train loss tot = 0.0367194,train loss reco = 0.0356031,train loss kl = 111.6627045:  26%|██▌       | 2/7.8125 [00:00<00:00,  6.60it/s]











train loss tot = 0.0367194,train loss reco = 0.0356031,train loss kl = 111.6627045:  38%|███▊      | 3/7.8125 [00:00<00:00,  6.73it/s]











train loss tot = 0.0478457,train loss reco = 0.0466607,train loss kl = 118.5424271:  38%|███▊      | 3/7.8125 [0

Epoch: 78, Training Loss Tot, Reco, KL :  0.0462,0.0451, 114.4302














train loss tot = 0.0519035,train loss reco = 0.0506676,train loss kl = 123.6405334:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.55it/s]











train loss tot = 0.0519035,train loss reco = 0.0506676,train loss kl = 123.6405334:  13%|█▎        | 1/7.8125 [00:00<00:01,  6.55it/s]











train loss tot = 0.0519035,train loss reco = 0.0506676,train loss kl = 123.6405334:  26%|██▌       | 2/7.8125 [00:00<00:00,  6.61it/s]











train loss tot = 0.0634801,train loss reco = 0.0623953,train loss kl = 108.5480652:  26%|██▌       | 2/7.8125 [00:00<00:00,  6.61it/s]











train loss tot = 0.0634801,train loss reco = 0.0623953,train loss kl = 108.5480652:  26%|██▌       | 2/7.8125 [00:00<00:00,  6.61it/s]











train loss tot = 0.0634801,train loss reco = 0.0623953,train loss kl = 108.5480652:  38%|███▊      | 3/7.8125 [00:00<00:00,  6.67it/s]











train loss tot = 0.0400923,train loss reco = 0.0389243,train loss kl = 116.8393021:  38%|███▊      | 3/7.8125 [0

Epoch: 79, Training Loss Tot, Reco, KL :  0.0479,0.0468, 110.0666


In [54]:
reload(utils_torch.scaler)
reload(prepr)
reload(plot)

save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
inverse_standardization = True
plot.plot_reco_for_loader(model, dataloaders['train'], device, scaler, inverse_standardization, 'test_train', osp.join(save_dir, 'reconstruction_post_train', 'train_reco_std_log_minmax_vae'), 'all')


In [34]:
input_fts, reco_fts = plot.gen_in_out(model, dataloaders['train'], device)


In [79]:
def t():
    a,b,c,d = 1,2,3,4
    return a,b,c,d

In [80]:
a = t()

In [83]:
len(a)

4